In [2]:
import tensorflow as tf
import numpy as np
import pandas
import keras
import math
from PIL import Image
import cv2,time
from keras.models import Sequential,load_model
from keras.layers import Dense, Activation,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Using TensorFlow backend.


In [9]:
model = load_model("C:\\Users\\Bhavesh\\Documents\\snapchat_filter\\weights.4C_F_3D.200.hdf5")

In [5]:
face_cascade = cv2.CascadeClassifier('C:\\Users\\Bhavesh\\Downloads\\haarcascade_frontalface_default.xml')

In [23]:
video=cv2.VideoCapture(0)
d=1
g=0
while True:
    d=d+1
    g=0
    check,frame=video.read()
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray_frame,1.5,5)
    for x,y,w,h in faces:
        crop_img = frame[y:y+h, x:x+w]
        img = gray_frame[y:y+h,x:x+w]
        g=1
    if g==1:
        img=cv2.resize(img,(96,96))#,Image.ANTIALIAS)
        test_img=np.array(img)/255
        test_img = np.reshape(test_img, (1,96,96,1))    # Model input shape = [batch_size, height, width, no. of channels]
        key_prediction = model.predict(test_img)      # shape = [batch_size, values]
        c=np.reshape(key_prediction,(15,2))
        a=x+(c[:,0]+0.5)*w
        b=y+(c[:,1]+0.5)*h
        a=a.astype(int)
        b=b.astype(int)
        frame[b,a]=[255,255,255]
    ##     FILTER-GLASS    #####
        glass=cv2.imread("C:\\Users\\Bhavesh\\Desktop\\glass.png")
        glass_width=(a[5]-a[3])*1.2
        glass_height=glass_width/4
        glass_width=glass_width.astype(int)
        glass_height=glass_height.astype(int)
        top_left=(a[3],b[3])                
        bottom_right=(a[5]+glass_width,b[5]+glass_height)             
        glass = cv2.resize(glass, (glass_width,glass_height))                                #ACTUAL GLASS LOADED FROM THE DISK
        glass_gray=cv2.cvtColor(glass,cv2.COLOR_BGR2GRAY)
        _,glass_mask= cv2.threshold(glass_gray,25,255,cv2.THRESH_BINARY_INV)
        area=frame[top_left[1]:top_left[1]+glass_height,top_left[0]:top_left[0]+glass_width] #AREA WHERE GLASS IS TO BE PLACED
        area_no_glass=cv2.bitwise_and(area,area,mask=glass_mask)
        final=cv2.add(area_no_glass,glass)
    
        frame[top_left[1]:top_left[1]+glass_height,top_left[0]:top_left[0]+glass_width]=final
    #gw,gh,gc = glass.shape
    #for i in range(0,gw):       # Overlay the filter based on the alpha channel
     #          for j in range(0,gh):
      #              if glass[i,j][3] != 0:
    #                   crop_img[brow_coords[1]+j-50,left_eye_coords[0]+i-90] = glass[i,j][:3]###############3
    ###############################
    cv2.imshow("pls",frame)
    key=cv2.waitKey(1)
    if key==ord('q'):
        break
print(d)
video.release()
cv2.destroyAllWindows()

732


In [22]:
print(a)
print(b)
print(glass_width)
print(glass_height)

[318 383 329 306 372 395 334 295 365 406 354 328 381 354 355]
[145 141 145 147 143 142 127 130 126 124 180 215 212 207 225]
106
26


In [16]:
print(a[5])
print(a[3])
print(glass_width/1.4)
print(a[5]-a[3])

376
293
-82.85714285714286
83
